#Solution for classification of documents


##1.1 Explanation
We would be using various algorithms to try and classify our classes between letters, memos and forms

As usual, we have to get our extracted data fro google drive 

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.zip'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1lJYTmX_PUyUVDSWvAiFkmoEJWZNA3msH'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('extractedData.zip')

In [0]:
!unzip extractedData.zip

In [0]:
import os

path = os.getcwd()  
print ("The current working directory is {}" .format(path))

In [0]:
classes = ["Email","Letter","Memo"]; 
dataFolders = ["train","test","val"];

In [0]:
import shutil
import os
import random
from pathlib import Path
try:
    from PIL import Image
except ImportError:
    import Image

In [0]:

import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

##1.2 Machine learning algorithms
We would be using the naive bayes algorithm and the SGD classifier  to classify the documents



In [0]:
# For reproducibility
np.random.seed(1237)
 
# train file directory
path_train = os.path.join(path, "extractedData", "train")
 
files_train = skds.load_files(path_train,load_content=False)
 
label_index = files_train.target
label_names = files_train.target_names
labelled_files = files_train.filenames

 
data_tags = ["filename","category","text"]
data_list = []
 
# Read and add data from file to a list
i=0
for f in labelled_files:
    data_list.append((f,label_names[label_index[i]],Path(f).read_text()))
    i += 1
 
# # We have training data available as dictionary filename, category, data
extractedtrainData = pd.DataFrame.from_records(data_list, columns=data_tags)


In [10]:

# test file directory
path_test = os.path.join(path, "extractedData", "test")
 
files_test = skds.load_files(path_test,load_content=False)
 
test_label_index = files_test.target
test_label_names = files_test.target_names
test_labelled_files = files_test.filenames


test_data_text = []
 
# Read and add data from file to a list

for f in test_labelled_files:
    test_data_text.append(Path(f).read_text())
    i += 1
    
print(test_labelled_files[0])
print(test_label_names)
print(len(test_data_text))

/content/extractedData/test/Letter/10230391.txt
['Email', 'Letter', 'Memo']
268


In [0]:
extractedtrainData.head()

,filename,category,text
0,/content/extractedData/train/Letter/517783754+...,Letter,@\n\n \n\n».DEC-04-97 THU 11:17 AM NB BULK PAC...
1,/content/extractedData/train/Memo/0000936338.txt,Memo,"Ts oe were eo rrr * Po Mi ""\n/\n™ x a ‘ a\nTE ..."
2,/content/extractedData/train/Email/2083182848d...,Email,"— Original Message-----\n\nFrom: Dawson, Rober..."
3,/content/extractedData/train/Memo/04107922_041...,Memo,": Lorillard\n\nMEMORANDUM\nJuly 27, 1982\nTO: ..."
4,/content/extractedData/train/Email/2070106870a...,Email,"oo Original Message-----\n\nFrom: Arce, Miguel..."


In [11]:
train_text = extractedtrainData['text']
train_tags = extractedtrainData['category']
train_files_names = extractedtrainData['filename']

train_text.shape

(1249,)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_text)
vectors.shape

(1249, 25752)

In [13]:
# using the example from sklearn

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


clf = MultinomialNB(alpha=.01)
clf.fit(vectors, label_index)

# newsgroups_test = fetch_20newsgroups(subset='test')
vectors_test = vectorizer.transform(test_data_text)

pred = clf.predict(vectors_test)
# np.mean(pred == newsgroups_test.target)
metrics.f1_score(test_label_index, pred, average='macro')

0.8841200077471059

In [14]:
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('tfidf', TfidfVectorizer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=10, random_state=42))])

text_clf_svm = text_clf_svm.fit(train_text, label_index)
predicted_svm = text_clf_svm.predict(test_data_text)

np.mean(predicted_svm == test_label_index)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


0.9029850746268657

Save the sgd classifier model and test saved model

In [17]:
pickle.dump(text_clf_svm, open('model.pkl','wb'))

model = pickle.load(open('model.pkl','rb'))
print(model.predict([validation_sentence]))

[2]


##1.3 Prediction using the validation test set

In [16]:

path_val = os.path.join(path, "extractedData", "val")
path_val

files_val = skds.load_files(path_val,load_content=False)
 
val_label_index = files_val.target
val_label_names = files_val.target_names
val_labelled_files = files_val.filenames


print(val_labelled_files[0])
print(val_label_names)

validation_sentence = Path(val_labelled_files[0]).read_text()
validation_sentence

vectors_val = vectorizer.transform([validation_sentence])

# Prediction for the naive bayes classifier
pred = clf.predict(vectors_val)
print(pred)

# Prediction for the SGD classifier
predicted_svm = text_clf_svm.predict([validation_sentence])
print(predicted_svm)

/content/extractedData/val/Memo/85418261.txt
['Email', 'Letter', 'Memo']
[2]
[2]


##1.4 Deep learning methodologies for classifciation

In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

Using TensorFlow backend.


In [0]:
# For reproducibility
np.random.seed(1237)
 
# train file directory
path_train = os.path.join(path, "extractedData", "train")
 
files_train = skds.load_files(path_train,load_content=False)
 
train_label_index = files_train.target
train_label_names = files_train.target_names
train_labelled_files = files_train.filenames

 
train_labelled_text = []

# Read and add data from file to a list
i=0

for file in train_labelled_files:
  with open(file, 'r') as f:
    text = f.read() 
    train_labelled_text.append(text)
  i += 1
    
print(len(train_label_index))
print(train_label_index[:10])

print(train_label_names)
print(train_labelled_text[:10])


1249
[1 2 0 2 0 2 0 0 2 2]
['Email', 'Letter', 'Memo']
["@\n\n \n\n\xc2\xbb.DEC-04-97 THU 11:17 AM NB BULK PACKAGING FAX NO. 7704293533 P, 05/08\n\nUnited States Food Safety Washington, 0.C.\nDepartment of and Inspection 20250\nAgriculture Service\n\nEgg Products Inspection Division\nJune 14, 1996\n\nMr. Gary Brown\nMacMillan Bloedel Bulk Packaging\nDivision of MacMillan Bloedel of America, Inc.\n\n1811 West Oak Parkway\nMarietta, Georgia 30062\n\nDear Mr, Brown:\n\nThis is in response to your June 4 letter requesting Department of Agriculmre (USDA)\napproval of the polypropylene Tre Blue Ball Valve, manufactured by BLDL, Inc, Totowa,\nNew Jersey, for use in plants operating under the federal egg products inspection program.\nYou propose to use the apparatus as an outlet valve for attachment to your firm's single-use\ncorrugated fiber IBC portable container approved for use in official egg products plants,\nEnclosed with your letter were specifications and drawings describing the const

##1.5 Extracting keywords from a file


In [0]:
NERTestFilePath = '/content/extractedData/test/Email/2067227128.txt'
with open(NERTestFilePath, 'r') as f:
    sentence = f.read() 

print(sentence)

Walk, Roger A.

From: Walk, Roger A.

Sent: Monday, July 22, 2002 12:27 PM
To: Roethig, Hans; Koller, Debbie
Subject: SCoR, clinical study 09/02

Dear Hans and Debbie,

In order to be prepared to any inquiries about this study to start in September with white packs of ‘U5’, Mike Pfeil suggests

that a holding statement be prepared. Could you give me your facts input such as

e Start and end of clinical trial

Number and characteristics of subjects (e.g., gender, race, voluntary and reimbursed, pregnancy, age)
Locations of trial

CRO and principal investigator

How is addressed that subjects are not exposed to increased risks

How does the test smoking relate to the ‘normal’ smoking of the subjects?

What is the purpose of the study?

By the way: What arrangement is contracted with the GRO in case they obtain media inquiries?

Regards,
Roger

8Z122zz90z


In [0]:
!pip install -U spacy

     |████████████████████████████████| 29.8MB 7.3MB/s 
     |████████████████████████████████| 184kB 46.6MB/s 
     |████████████████████████████████| 3.2MB 38.4MB/s 
     |████████████████████████████████| 2.1MB 39.9MB/s 
  Found existing installation: thinc 6.12.1
    Uninstalling thinc-6.12.1:
      Successfully uninstalled thinc-6.12.1
  Found existing installation: spacy 2.0.18
    Uninstalling spacy-2.0.18:
      Successfully uninstalled spacy-2.0.18


In [0]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 11.1MB 608kB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqjck4np/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.0.0
    Uninstalling en-core-web-sm-2.0.0:
      Successfully uninstalled en-core-web-sm-2.0.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint

In [0]:
doc = nlp(sentence)
##pprint([(X.text, X.label_) for X in doc.ents])
pprint([(X.text) for X in doc.ents])

['Roger A.\n\n',
 'Walk',
 'Roger A.\n\nSent',
 'Monday, July 22, 2002',
 '12:27 PM',
 'Hans',
 'Koller',
 'Debbie',
 'SCoR',
 '09/02',
 'Debbie',
 'September',
 'Mike Pfeil',
 'GRO',
 'Roger\n\n8Z122zz90z']
